In [70]:
# # Do not forget to check if your notebook log collection plugin is working
from mining_extension import check_logging
check_logging("http://3.249.245.244:9999")

Request successful!


In [71]:
import catboost
import sklearn.feature_extraction.text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import sklearn
import numpy as np

In [72]:
import pandas as pd
features_path = 'data/task2/'
test_features = pd.read_pickle(features_path+'test_features.pkl')
train_features = pd.read_pickle(features_path+'train_features.pkl')
validation_features = pd.read_pickle(features_path+'validation_features.pkl')

train_features.index = range(train_features.shape[0])
validation_features.index = range(validation_features.shape[0])
test_features.index = range(test_features.shape[0])

train_features.fillna(0,inplace=True)
test_features.fillna(0,inplace=True)
validation_features.fillna(0,inplace=True)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(5833, 32)
(1927, 32)
(1918, 21)


In [73]:
train_features

,filename,cell_type,cell_number,execution_count,linesofcomment,linesofcode,variable_count,function_count,text/plain,image/png,...,helper_functions,load_data,data_exploration,data_preprocessing,evaluation,modelling,prediction,result_visualization,save_results,comment_only
0,nb_54880.ipynb,code,0,1,0.0,2.0,0.0,0.0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,nb_54880.ipynb,code,1,2,1.0,4.0,4.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,nb_54880.ipynb,code,2,3,0.0,1.0,0.0,0.0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,nb_54880.ipynb,code,3,4,0.0,1.0,0.0,0.0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,nb_54880.ipynb,code,4,5,0.0,1.0,1.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,nb_95821.ipynb,code,27,-1,0.0,2.0,1.0,0.0,0,0,...,0,0,0,0,1,0,0,0,0,0
5829,nb_95821.ipynb,code,29,-1,0.0,2.0,1.0,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
5830,nb_95821.ipynb,code,31,-1,0.0,3.0,2.0,0.0,0,0,...,0,0,1,1,0,0,0,0,0,0
5831,nb_95821.ipynb,code,33,-1,5.0,17.0,14.0,0.0,0,0,...,0,0,0,0,1,1,1,0,0,0


In [74]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5833 entries, 0 to 5832
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   filename              5833 non-null   object 
 1   cell_type             5833 non-null   object 
 2   cell_number           5833 non-null   int64  
 3   execution_count       5833 non-null   int64  
 4   linesofcomment        5833 non-null   float64
 5   linesofcode           5833 non-null   float64
 6   variable_count        5833 non-null   float64
 7   function_count        5833 non-null   float64
 8   text/plain            5833 non-null   int64  
 9   image/png             5833 non-null   int64  
 10  text/html             5833 non-null   int64  
 11  execute_result        5833 non-null   int64  
 12  display_data          5833 non-null   int64  
 13  stream                5833 non-null   int64  
 14  error                 5833 non-null   int64  
 15  text                 

In [75]:
train_features.dtypes[3]

dtype('int64')

According to columns description I made a decision to divide all the columns by type

In [76]:
cat_cols = ['cell_type']

In [77]:
one_hot_cols = ['error','stream','display_data', 'text/plain', 'image/png', 'text/html', 'execute_result']

In [78]:
num_cols = ['cell_number', 'execution_count', 'linesofcomment', 'linesofcode', 'variable_count']

In [79]:
text_cols = ['comment', 'filename', 'text', 'code_line_before', 'code_line_after', 'markdown_heading', 'packages_info']

In [80]:
train_features = train_features.drop(text_cols, axis = 1)

In [81]:
train_labels = train_features['primary_label']
train_labels

0         helper_functions
1                load_data
2         data_exploration
3         data_exploration
4       data_preprocessing
               ...        
5828            evaluation
5829             modelling
5830    data_preprocessing
5831             modelling
5832            evaluation
Name: primary_label, Length: 5833, dtype: object

In [84]:
cbc = CatBoostClassifier(custom_metric='F1', boosting_type = 'Plain',loss_function='MultiClass', score_function='Cosine')

In [85]:
train_d = train_features.drop(['primary_label'], axis = 1)
# test_d = test_features.drop(['primary_label'], axis = 1)
validation_d = validation_features.drop(['primary_label']+text_cols, axis = 1)

train_labels = train_labels
# test_labels = test_features['primary_label']
validation_labels = validation_features['primary_label']


from catboost import Pool

train_data = Pool(train_d,
    label = train_labels,
    cat_features=cat_cols+one_hot_cols
)

eval_data = Pool(validation_d,
    label = validation_labels,
    cat_features=cat_cols+one_hot_cols
)

cbc.fit(train_data, eval_set=eval_data)
preds_class = cbc.predict(eval_data)



Learning rate set to 0.111852
0:	learn: 1.5649552	test: 1.5605078	best: 1.5605078 (0)	total: 66.5ms	remaining: 1m 6s
1:	learn: 1.2713590	test: 1.2767667	best: 1.2767667 (1)	total: 125ms	remaining: 1m 2s
2:	learn: 1.0685985	test: 1.0788313	best: 1.0788313 (2)	total: 183ms	remaining: 1m
3:	learn: 0.9251845	test: 0.9349273	best: 0.9349273 (3)	total: 245ms	remaining: 1m 1s
4:	learn: 0.8147522	test: 0.8249850	best: 0.8249850 (4)	total: 304ms	remaining: 1m
5:	learn: 0.7209162	test: 0.7319931	best: 0.7319931 (5)	total: 361ms	remaining: 59.8s
6:	learn: 0.6435161	test: 0.6569806	best: 0.6569806 (6)	total: 420ms	remaining: 59.6s
7:	learn: 0.5800808	test: 0.5980796	best: 0.5980796 (7)	total: 487ms	remaining: 1m
8:	learn: 0.5244342	test: 0.5436475	best: 0.5436475 (8)	total: 549ms	remaining: 1m
9:	learn: 0.4749040	test: 0.4951427	best: 0.4951427 (9)	total: 610ms	remaining: 1m
10:	learn: 0.4377800	test: 0.4584671	best: 0.4584671 (10)	total: 669ms	remaining: 1m
11:	learn: 0.4038284	test: 0.4246917	be

94:	learn: 0.0697085	test: 0.1173763	best: 0.1173763 (94)	total: 8.35s	remaining: 1m 19s
95:	learn: 0.0691408	test: 0.1167411	best: 0.1167411 (95)	total: 8.46s	remaining: 1m 19s
96:	learn: 0.0691038	test: 0.1166562	best: 0.1166562 (96)	total: 8.58s	remaining: 1m 19s
97:	learn: 0.0681494	test: 0.1160212	best: 0.1160212 (97)	total: 8.66s	remaining: 1m 19s
98:	learn: 0.0675337	test: 0.1148674	best: 0.1148674 (98)	total: 8.75s	remaining: 1m 19s
99:	learn: 0.0673987	test: 0.1148072	best: 0.1148072 (99)	total: 8.85s	remaining: 1m 19s
100:	learn: 0.0667129	test: 0.1145841	best: 0.1145841 (100)	total: 8.94s	remaining: 1m 19s
101:	learn: 0.0665498	test: 0.1144509	best: 0.1144509 (101)	total: 9.04s	remaining: 1m 19s
102:	learn: 0.0660939	test: 0.1142236	best: 0.1142236 (102)	total: 9.13s	remaining: 1m 19s
103:	learn: 0.0656617	test: 0.1136674	best: 0.1136674 (103)	total: 9.23s	remaining: 1m 19s
104:	learn: 0.0649761	test: 0.1132152	best: 0.1132152 (104)	total: 9.32s	remaining: 1m 19s
105:	learn:

186:	learn: 0.0450779	test: 0.1031867	best: 0.1031867 (186)	total: 17.7s	remaining: 1m 16s
187:	learn: 0.0450280	test: 0.1031333	best: 0.1031333 (187)	total: 17.8s	remaining: 1m 16s
188:	learn: 0.0449744	test: 0.1031710	best: 0.1031333 (187)	total: 17.9s	remaining: 1m 16s
189:	learn: 0.0448719	test: 0.1030211	best: 0.1030211 (189)	total: 18s	remaining: 1m 16s
190:	learn: 0.0448401	test: 0.1030583	best: 0.1030211 (189)	total: 18.1s	remaining: 1m 16s
191:	learn: 0.0446782	test: 0.1036621	best: 0.1030211 (189)	total: 18.2s	remaining: 1m 16s
192:	learn: 0.0446227	test: 0.1036355	best: 0.1030211 (189)	total: 18.3s	remaining: 1m 16s
193:	learn: 0.0442952	test: 0.1035438	best: 0.1030211 (189)	total: 18.4s	remaining: 1m 16s
194:	learn: 0.0441607	test: 0.1030721	best: 0.1030211 (189)	total: 18.5s	remaining: 1m 16s
195:	learn: 0.0440637	test: 0.1028274	best: 0.1028274 (195)	total: 18.6s	remaining: 1m 16s
196:	learn: 0.0438590	test: 0.1023549	best: 0.1023549 (196)	total: 18.7s	remaining: 1m 16s
1

278:	learn: 0.0337617	test: 0.1012972	best: 0.1008888 (236)	total: 26.8s	remaining: 1m 9s
279:	learn: 0.0336932	test: 0.1013035	best: 0.1008888 (236)	total: 27s	remaining: 1m 9s
280:	learn: 0.0335347	test: 0.1013378	best: 0.1008888 (236)	total: 27.1s	remaining: 1m 9s
281:	learn: 0.0334304	test: 0.1015225	best: 0.1008888 (236)	total: 27.2s	remaining: 1m 9s
282:	learn: 0.0334028	test: 0.1015883	best: 0.1008888 (236)	total: 27.3s	remaining: 1m 9s
283:	learn: 0.0331779	test: 0.1014650	best: 0.1008888 (236)	total: 27.3s	remaining: 1m 8s
284:	learn: 0.0330779	test: 0.1015403	best: 0.1008888 (236)	total: 27.4s	remaining: 1m 8s
285:	learn: 0.0330708	test: 0.1015555	best: 0.1008888 (236)	total: 27.5s	remaining: 1m 8s
286:	learn: 0.0329762	test: 0.1014556	best: 0.1008888 (236)	total: 27.7s	remaining: 1m 8s
287:	learn: 0.0329269	test: 0.1014562	best: 0.1008888 (236)	total: 27.7s	remaining: 1m 8s
288:	learn: 0.0327704	test: 0.1015556	best: 0.1008888 (236)	total: 27.8s	remaining: 1m 8s
289:	learn: 

372:	learn: 0.0256446	test: 0.1021669	best: 0.1002816 (305)	total: 36.3s	remaining: 1m 1s
373:	learn: 0.0255911	test: 0.1019919	best: 0.1002816 (305)	total: 36.5s	remaining: 1m 1s
374:	learn: 0.0255486	test: 0.1020100	best: 0.1002816 (305)	total: 36.6s	remaining: 1m
375:	learn: 0.0254094	test: 0.1021563	best: 0.1002816 (305)	total: 36.7s	remaining: 1m
376:	learn: 0.0253682	test: 0.1020069	best: 0.1002816 (305)	total: 36.8s	remaining: 1m
377:	learn: 0.0252970	test: 0.1019355	best: 0.1002816 (305)	total: 36.9s	remaining: 1m
378:	learn: 0.0251906	test: 0.1020124	best: 0.1002816 (305)	total: 37s	remaining: 1m
379:	learn: 0.0251377	test: 0.1021142	best: 0.1002816 (305)	total: 37.1s	remaining: 1m
380:	learn: 0.0250734	test: 0.1020039	best: 0.1002816 (305)	total: 37.2s	remaining: 1m
381:	learn: 0.0249448	test: 0.1019924	best: 0.1002816 (305)	total: 37.3s	remaining: 1m
382:	learn: 0.0249353	test: 0.1020006	best: 0.1002816 (305)	total: 37.4s	remaining: 1m
383:	learn: 0.0248715	test: 0.1018366	b

...

557:	learn: 0.0170805	test: 0.1024840	best: 0.1002816 (305)	total: 58.5s	remaining: 46.3s
558:	learn: 0.0170617	test: 0.1025155	best: 0.1002816 (305)	total: 58.6s	remaining: 46.2s
559:	learn: 0.0170154	test: 0.1024845	best: 0.1002816 (305)	total: 58.7s	remaining: 46.2s
560:	learn: 0.0170012	test: 0.1024962	best: 0.1002816 (305)	total: 58.9s	remaining: 46.1s
561:	learn: 0.0169902	test: 0.1024968	best: 0.1002816 (305)	total: 59s	remaining: 46s
562:	learn: 0.0169673	test: 0.1025656	best: 0.1002816 (305)	total: 59.1s	remaining: 45.9s
563:	learn: 0.0169522	test: 0.1025664	best: 0.1002816 (305)	total: 59.2s	remaining: 45.8s
564:	learn: 0.0168904	test: 0.1027885	best: 0.1002816 (305)	total: 59.3s	remaining: 45.7s
565:	learn: 0.0168572	test: 0.1027375	best: 0.1002816 (305)	total: 59.4s	remaining: 45.6s
566:	learn: 0.0168410	test: 0.1028276	best: 0.1002816 (305)	total: 59.6s	remaining: 45.5s
567:	learn: 0.0167166	test: 0.1027547	best: 0.1002816 (305)	total: 59.6s	remaining: 45.4s
568:	learn: 0.

650:	learn: 0.0142394	test: 0.1039428	best: 0.1002816 (305)	total: 1m 9s	remaining: 37.5s
651:	learn: 0.0142195	test: 0.1039668	best: 0.1002816 (305)	total: 1m 10s	remaining: 37.4s
652:	learn: 0.0142151	test: 0.1039532	best: 0.1002816 (305)	total: 1m 10s	remaining: 37.3s
653:	learn: 0.0141850	test: 0.1037377	best: 0.1002816 (305)	total: 1m 10s	remaining: 37.2s
654:	learn: 0.0141745	test: 0.1038453	best: 0.1002816 (305)	total: 1m 10s	remaining: 37.1s
655:	learn: 0.0141556	test: 0.1038423	best: 0.1002816 (305)	total: 1m 10s	remaining: 37s
656:	learn: 0.0141287	test: 0.1038573	best: 0.1002816 (305)	total: 1m 10s	remaining: 36.9s
657:	learn: 0.0141094	test: 0.1038891	best: 0.1002816 (305)	total: 1m 10s	remaining: 36.8s
658:	learn: 0.0140631	test: 0.1038985	best: 0.1002816 (305)	total: 1m 10s	remaining: 36.7s
659:	learn: 0.0140580	test: 0.1039567	best: 0.1002816 (305)	total: 1m 11s	remaining: 36.6s
660:	learn: 0.0140541	test: 0.1039437	best: 0.1002816 (305)	total: 1m 11s	remaining: 36.5s
66

743:	learn: 0.0121382	test: 0.1051133	best: 0.1002816 (305)	total: 1m 21s	remaining: 27.9s
744:	learn: 0.0121177	test: 0.1052044	best: 0.1002816 (305)	total: 1m 21s	remaining: 27.8s
745:	learn: 0.0120790	test: 0.1050415	best: 0.1002816 (305)	total: 1m 21s	remaining: 27.7s
746:	learn: 0.0120687	test: 0.1051145	best: 0.1002816 (305)	total: 1m 21s	remaining: 27.6s
747:	learn: 0.0120502	test: 0.1051557	best: 0.1002816 (305)	total: 1m 21s	remaining: 27.5s
748:	learn: 0.0120362	test: 0.1051082	best: 0.1002816 (305)	total: 1m 21s	remaining: 27.4s
749:	learn: 0.0120319	test: 0.1051223	best: 0.1002816 (305)	total: 1m 21s	remaining: 27.3s
750:	learn: 0.0120251	test: 0.1050723	best: 0.1002816 (305)	total: 1m 21s	remaining: 27.2s
751:	learn: 0.0120210	test: 0.1052077	best: 0.1002816 (305)	total: 1m 22s	remaining: 27.1s
752:	learn: 0.0120010	test: 0.1051652	best: 0.1002816 (305)	total: 1m 22s	remaining: 27s
753:	learn: 0.0119813	test: 0.1052575	best: 0.1002816 (305)	total: 1m 22s	remaining: 26.9s
7

835:	learn: 0.0103928	test: 0.1067426	best: 0.1002816 (305)	total: 1m 32s	remaining: 18.1s
836:	learn: 0.0103555	test: 0.1068756	best: 0.1002816 (305)	total: 1m 32s	remaining: 18s
837:	learn: 0.0103237	test: 0.1068997	best: 0.1002816 (305)	total: 1m 32s	remaining: 17.9s
838:	learn: 0.0103066	test: 0.1069335	best: 0.1002816 (305)	total: 1m 32s	remaining: 17.8s
839:	learn: 0.0102964	test: 0.1068954	best: 0.1002816 (305)	total: 1m 32s	remaining: 17.7s
840:	learn: 0.0102652	test: 0.1068711	best: 0.1002816 (305)	total: 1m 32s	remaining: 17.6s
841:	learn: 0.0102492	test: 0.1068503	best: 0.1002816 (305)	total: 1m 33s	remaining: 17.5s
842:	learn: 0.0102365	test: 0.1069274	best: 0.1002816 (305)	total: 1m 33s	remaining: 17.3s
843:	learn: 0.0102315	test: 0.1069599	best: 0.1002816 (305)	total: 1m 33s	remaining: 17.2s
844:	learn: 0.0102222	test: 0.1069125	best: 0.1002816 (305)	total: 1m 33s	remaining: 17.1s
845:	learn: 0.0102055	test: 0.1070047	best: 0.1002816 (305)	total: 1m 33s	remaining: 17s
846

926:	learn: 0.0091434	test: 0.1090421	best: 0.1002816 (305)	total: 1m 42s	remaining: 8.11s
927:	learn: 0.0091308	test: 0.1090804	best: 0.1002816 (305)	total: 1m 43s	remaining: 8s
928:	learn: 0.0091221	test: 0.1090412	best: 0.1002816 (305)	total: 1m 43s	remaining: 7.89s
929:	learn: 0.0091098	test: 0.1089647	best: 0.1002816 (305)	total: 1m 43s	remaining: 7.78s
930:	learn: 0.0090834	test: 0.1089133	best: 0.1002816 (305)	total: 1m 43s	remaining: 7.67s
931:	learn: 0.0090565	test: 0.1089428	best: 0.1002816 (305)	total: 1m 43s	remaining: 7.55s
932:	learn: 0.0090484	test: 0.1089393	best: 0.1002816 (305)	total: 1m 43s	remaining: 7.44s
933:	learn: 0.0090412	test: 0.1089440	best: 0.1002816 (305)	total: 1m 43s	remaining: 7.33s
934:	learn: 0.0090360	test: 0.1089576	best: 0.1002816 (305)	total: 1m 43s	remaining: 7.22s
935:	learn: 0.0090310	test: 0.1089971	best: 0.1002816 (305)	total: 1m 44s	remaining: 7.12s
936:	learn: 0.0090192	test: 0.1090100	best: 0.1002816 (305)	total: 1m 44s	remaining: 7s
937:	